In [33]:
import json
import datetime

In [14]:
%run lib.ipynb

In [15]:
with open( "./data/playoff-schedule.json") as f:
    schedule_data = json.load(f)

schedule = DictObj(schedule_data)

### Game Parser

In [29]:
def parseGameById( gameFileId ):
        
    # gameFileId = '8f36d70b-a463-452e-9f02-65dd5d37d2c9'

    with open( f"./output/games/game_{gameFileId}.json") as f:
        data = json.load(f)

    _gameObj = DictObj(data)
    
    home_reference = _gameObj.home.reference
    away_reference = _gameObj.away.reference
    
    report = {
        'periods': _gameObj.period,
        'teams':{
            'home': _gameObj.home.name,
            'away': _gameObj.away.name,
        },
        'score':{
            'home': _gameObj.home.points,
            'away': _gameObj.away.points,
        },
        'shots':{
            'home' : {
                'shotsaved': 0,
                'shotmissed': 0
            },
            'away' : {
                'shotsaved': 0,
                'shotmissed': 0
            },
        }
        
    }
    
    home_shotssaved = 0
    home_shotmissed = 0
    
    away_shotssaved = 0
    away_shotmissed = 0
    
    # periods - events
    for period in _gameObj.periods:
        for event in period.events:
            
            if( event.event_type != "shotmissed" and  event.event_type != "shotsaved" ):
                continue
                    
            if(event.attribution.reference == home_reference):
                if( event.event_type == "shotsaved"  ):
                    home_shotssaved += 1
                if( event.event_type == "shotmissed" ):
                    home_shotmissed += 1
            else:
                if( event.event_type == "shotsaved"  ):
                    away_shotssaved += 1
                if( event.event_type == "shotmissed" ):
                    away_shotmissed += 1
                    
    report['shots']['home']['shotsaved'] = home_shotssaved
    report['shots']['home']['shotmissed'] = home_shotmissed
    report['shots']['away']['shotsaved'] = away_shotssaved
    report['shots']['away']['shotmissed'] = away_shotmissed

    
    return report
    
    

In [30]:
r = parseGameById('4c9a706e-b53d-484c-95a3-a9cd7105bc63')
print(r)

{'periods': 3, 'teams': {'home': 'Hurricanes', 'away': 'Rangers'}, 'score': {'home': 3, 'away': 5}, 'shots': {'home': {'shotsaved': 33, 'shotmissed': 27}, 'away': {'shotsaved': 19, 'shotmissed': 43}}}


### Loop

In [36]:
def getRoundGroupData( groupItem, round ):
    
    teams = groupItem.teams
    team_a = teams[0]
    team_b = teams[1]
    
    _groupReport = {
        'team_a': team_a,
        'team_b': team_b,
        'round': round,
        'games_reports':[]
    }
        
    num_games = len(groupItem.games)
    
    for gameItem in groupItem.games:
        game_score = gameItem.score.split(':')
        
        score_a = game_score[0]
        score_b = game_score[1]

        game_id = gameItem.id
        
        reportData = parseGameById( game_id )
        
        dateNums = gameItem.date.split('.')
        x = datetime.datetime(2024, int(dateNums[1]), int(dateNums[0]))
        dateFormated = x.strftime("%Y-%m-%d")
        
        _reportObj = {
            'schedule_data': {
                'game': gameItem.game,
                'id': gameItem.id,
                'date': gameItem.date,
                'date_format': dateFormated
            },
            'report': reportData
        }
        
        _groupReport['games_reports'].append(_reportObj)
        
    file_name = f"./output/reports/widget-shots-total/widget-shots-total_{team_a}-{team_b}.json"
    with open(file_name, "w") as outfile:
        json.dump(_groupReport, outfile, indent=2)
    
    return _groupReport

# Init

In [39]:
# Change this and run:  0,1,2
roundIdx = 0 

roundItem = schedule.rounds[roundIdx]
round = roundItem.round

for groupItem in roundItem.groups:
    group_report = getRoundGroupData(groupItem, round)